# LangChain Expression Language (LCEL)

The **L**ang**C**hain **E**xpression **L**anguage (LCEL) abstracts key Python concepts into a streamlined format, facilitating a "minimalist" code layer for constructing chains of LangChain components. LCEL offers robust support for:

1. Rapid development of chains.
2. Advanced features like streaming, asynchronous processing, parallel execution, and more.
3. Seamless integration with LangSmith and LangServe.

In [1]:

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_ollama import ChatOllama
from rich import print as pp

# Create a prompt template for generating book summaries
prompt = ChatPromptTemplate.from_template(
    "Summarize the book titled '{book_title}' in three sentences."
)

# Use a specific LLM model
model = ChatOllama(model="llama3.2")


## LCEL Syntax

In [2]:

output_parser = StrOutputParser()

from langchain.chains import LLMChain

# Create a chain for summarizing books
chain = LLMChain(
    prompt=prompt,
    llm=model,
    output_parser=output_parser
)


/var/folders/m7/yjmck8kn59gc9w3kdklj2lt40000gn/T/ipykernel_52966/2803915055.py:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [3]:

# Run the chain
out = chain.run(book_title="1984 by George Orwell")
print(out)


/var/folders/m7/yjmck8kn59gc9w3kdklj2lt40000gn/T/ipykernel_52966/355639390.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  out = chain.run(book_title="1984 by George Orwell")


Here is a summary of "1984" by George Orwell:

In the dystopian novel "1984," the totalitarian government of Oceania, led by a figure known as Big Brother, exercises total control over its citizens, suppressing individual freedom and independent thought through propaganda, surveillance, and manipulation. The protagonist, Winston Smith, a low-ranking member of the ruling Party, begins to question the official ideology and starts an illicit love affair with a fellow worker, Julia, which ultimately leads to their discovery by the authorities and their subsequent torture and brainwashing. Ultimately, Winston is driven mad by the relentless pursuit of truth and individuality, and he surrenders his independence to Big Brother, exemplifying the chilling notion that "war is peace," "freedom is slavery," and "ignorance is strength."


In [4]:

# Alternative LCEL-style chain
lcel_chain = prompt | model | output_parser

# Run the chain
out = lcel_chain.invoke({"book_title": "To Kill a Mockingbird"})
print(out)


Here is a summary of "To Kill a Mockingbird" in three sentences:

Set in the Deep South during the 1930s, the novel follows the experiences of Scout Finch, a young girl who lives with her older brother Jem and their father, Atticus, in the fictional town of Maycomb. When a black man named Tom Robinson is falsely accused of raping a white woman, Atticus agrees to defend him in court despite knowing he'll face prejudice and hostility, teaching Scout and Jem valuable lessons about justice, morality, and empathy. Ultimately, the trial ends in tragedy when the all-white jury delivers a guilty verdict, but Atticus's defense and teachings have a lasting impact on his children and the community, leaving them with a newfound understanding of the importance of treating all people with kindness and respect.


## How the Pipe Operator Works
To truly grasp LCEL, let's examine how the pipe operation functions. It takes output from the **right** and feeds it to the **left**—but since this isn't standard Python, how is it implemented? We can create our own version using simple functions.

We'll utilize the `__or__` method in Python class objects. When we combine two classes like `chain = class_a | class_b`, the Python interpreter checks for the presence of the `__or__` method in these classes. If it exists, the expression `|` is translated to `chain = class_a.__or__(class_b)`. 

This means both of the following patterns yield the same result:

```python
# Object approach
chain = class_a.__or__(class_b)
chain("some input")

# Pipe approach
chain = class_a | class_b
chain("some input")
```

With this understanding, we can create a `Runnable` class that takes a function and transforms it into a chainable function using the pipe operator `|`.



In [5]:

class Runnable:
    def __init__(self, func):
        self.func = func

    def __or__(self, other):
        def chained_func(*args, **kwargs):
            return other(self.func(*args, **kwargs))
        return Runnable(chained_func)

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)


def subtract_three(x):
    return x - 3


def divide_by_two(x):
    return x / 2


subtract_three = Runnable(subtract_three)
divide_by_two = Runnable(divide_by_two)

chain = subtract_three | divide_by_two
print(chain(10))


3.5


## LCEL Deep Dive
Now that we understand what this syntax is doing under the hood, let's explore it within the context of LCEL and see a few of the additional methods that LangChain has provided to maximize flexibility when working with LCEL.

In [7]:

from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

# Create an embedding model
embedding = OllamaEmbeddings(model="snowflake-arctic-embed:33m") 
# Create vector stores with different content
vecstore_a = InMemoryVectorStore.from_texts(
    ["Python is a programming language.", "It is popular for data science."],
    embedding=embedding
)
vecstore_b = InMemoryVectorStore.from_texts(
    ["Python supports object-oriented programming.", "It is used for web development."],
    embedding=embedding
)


Here we have used `RunnableParallel` to create two parallel streams of information, one for `"context"` that is information fed in from `retriever_a`, and another for `"question"` which is the _passthrough_ information, ie the information that is passed through from our `chain.invoke("when was James born?")` call.

Using this information the chain is close to answering the question but it doesn't have enough information, it is missing the information that we have stored in `retriever_b`. Fortunately, we can have multiple parallel information streams using the `RunnableParallel` object.

In [8]:

from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)

retriever_a = vecstore_a.as_retriever()
retriever_b = vecstore_b.as_retriever()

prompt_str = """Provide details about Python:

Context: {context}

Question: {question}

Answer:"""

prompt = ChatPromptTemplate.from_template(prompt_str)

retriever = RunnableParallel(
    {'context': retriever_a, 'question': RunnablePassthrough()},
)

chain = retriever | prompt | model | output_parser

pp(chain)

out = chain.invoke("What are the uses of Python?")
print(out)


RunnableSequence(
    first=RunnableParallel(
        steps__={
            'context': VectorStoreRetriever(
                tags=['InMemoryVectorStore', 'OllamaEmbeddings'],
                vectorstore=<langchain_core.vectorstores.in_memory.InMemoryVectorStore object at 0x12ad98c90>,
                search_kwargs={}
            ),
            'question': RunnablePassthrough()
        }
    ),
    middle=[
        ChatPromptTemplate(
            input_variables=['context', 'question'],
            input_types={},
            partial_variables={},
            messages=[
                HumanMessagePromptTemplate(
                    prompt=PromptTemplate(
                        input_variables=['context', 'question'],
                        input_types={},
                        partial_variables={},
                        template='Provide details about Python:\n\nContext: {context}\n\nQuestion: 
{question}\n\nAnswer:'
                    ),
                    additional_kwargs={}
                )
            ]
        ),
        ChatOllama(model='llama3.2')
    ],
    last=StrOutputParser()
)

Python is a versatile and widely-used programming language that has numerous applications in various fields. Some of the main uses of Python include:

1. **Data Science and Machine Learning**: Python is a popular choice for data scientists and machine learning engineers due to its extensive libraries such as NumPy, pandas, and scikit-learn, which provide efficient data structures and algorithms for data analysis and modeling.
2. **Web Development**: Python can be used for web development with frameworks like Django and Flask, which enable rapid development of secure and scalable web applications.
3. **Automation**: Python's easy-to-learn syntax and vast number of libraries make it an ideal language for automating tasks, such as data processing, file management, and system administration.
4. **Scientific Computing**: Python is widely used in scientific computing for tasks like numerical analysis, visualization, and simulation due to its integration with libraries like SciPy, NumPy, and 

## Runnable Lambdas
The `RunnableLambda` is a LangChain abstraction that allows us to turn Python functions into pipe-compatible function, similar to the `Runnable` class we created near the beginning of this notebook.

Let's try it out with our earlier `add_five` and `multiply_by_two` functions.

In [9]:

def calc_cube(x):
    return x ** 3

def calc_reciprocal(x):
    return 1 / x

from langchain_core.runnables import RunnableLambda

calc_cube = RunnableLambda(calc_cube)
calc_reciprocal = RunnableLambda(calc_reciprocal)

chain = calc_cube | calc_reciprocal
pp(chain)

result = chain.invoke(4)
print(result)


RunnableSequence(first=RunnableLambda(calc_cube), middle=[], last=RunnableLambda(calc_reciprocal))

0.015625


In [10]:

def extract_first_sentence(x):
    return x.split(".")[0] + "."

get_first_sentence = RunnableLambda(extract_first_sentence)

prompt_str = "Share an interesting fact about {topic}."
prompt = ChatPromptTemplate.from_template(prompt_str)

chain = prompt | model | output_parser | get_first_sentence
pp(chain)

fact = chain.invoke({"topic": "Machine Learning"})
print(fact)


RunnableSequence(
    first=ChatPromptTemplate(
        input_variables=['topic'],
        input_types={},
        partial_variables={},
        messages=[
            HumanMessagePromptTemplate(
                prompt=PromptTemplate(
                    input_variables=['topic'],
                    input_types={},
                    partial_variables={},
                    template='Share an interesting fact about {topic}.'
                ),
                additional_kwargs={}
            )
        ]
    ),
    middle=[ChatOllama(model='llama3.2'), StrOutputParser()],
    last=RunnableLambda(extract_first_sentence)
)

Here's an interesting fact about Machine Learning:

Did you know that the concept of Machine Learning was first introduced by Arthur Samuel in 1956? He proposed a rule-based system called "Checkers" which could learn to play the game through self-play and improvement.
